In [1]:
def IntialiseEnv():
    nb_dir = os.path.split(os.getcwd())[0]
    if nb_dir not in sys.path:
        sys.path.append(nb_dir)
    invalid_path='/opt/ros/kinetic/lib/python2.7/dist-packages'
    if invalid_path in sys.path:
        sys.path.remove(invalid_path)

In [2]:
# Libraries
import os
import sys
IntialiseEnv()
import tensorflow as tf
import numpy as np
import math
import import_ipynb
from sklearn.model_selection import train_test_split
from Data_Preparation.Data_Preparation import LoadTrainDataSet

importing Jupyter notebook from /home/rana/Documents/Deep_Learning/Project/Traffic_Sign_Recognition_Detection/Source_Code/Data_Preparation/Data_Preparation.ipynb
importing Jupyter notebook from /home/rana/Documents/Deep_Learning/Project/Traffic_Sign_Recognition_Detection/Source_Code/Feature_Extraction/Segmentation.ipynb


In [3]:
# Use GPU/CPU Configurations

In [4]:
class CNN_Model(tf.keras.Model):
    def __init__(self):
        super(CNN_Model, self).__init__()
        initializer = tf.initializers.VarianceScaling(scale=2.0)
        self.conv_layer_1 = tf.keras.layers.Conv2D(filter_size,kernel_1, strides = 1, padding='same',activation = tf.nn.relu)
        self.pool_layer_1 = tf.keras.layers.MaxPool2D(pooling)
        self.conv_layer_2 = tf.keras.layers.Conv2D(filter_size,kernel_2, strides = 1, padding='same', activation = tf.nn.relu)
        self.pool_layer_2 = tf.keras.layers.MaxPool2D(pooling)
        self.conv_layer_3 = tf.keras.layers.Conv2D(filter_size,kernel_2, strides = 1, padding='same', activation = tf.nn.relu)
        self.pool_layer_3 = tf.keras.layers.MaxPool2D(pooling)
        self.flatten = tf.keras.layers.Flatten()
        self.fc = tf.keras.layers.Dense(hidden_layer, activation = tf.nn.relu)
        self.drop = tf.keras.layers.Dropout(dropout)
        self.final = tf.keras.layers.Dense(num_classes, activation = 'softmax')
    
    def call(self, input_tensor, training=False):
        x = self.conv_layer_1(input_tensor)
        x = self.pool_layer_1(x)
        x = self.conv_layer_2(x)
        x = self.pool_layer_2(x)        
        x = self.conv_layer_3(x)
        x = self.pool_layer_3(x)
        x = self.flatten(x)
        x = self.fc(x)
        x = self.drop(x)
        x = self.final(x)
        return x

In [ ]:
def train(model_init_fn, optimizer_init_fn, num_epochs=1,TrainDataSet, is_training=False):  

        # Compute the loss like we did in Part II
        loss_fn = tf.keras.losses.SparseCategoricalCrossentropy()
        
        model = model_init_fn()
        optimizer = optimizer_init_fn()
        
        train_loss = tf.keras.metrics.Mean(name='train_loss')
        train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')
    
        val_loss = tf.keras.metrics.Mean(name='val_loss')
        val_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='val_accuracy')
        
        t = 0
        for epoch in range(num_epochs):
            
            train_loss.reset_states()
            train_accuracy.reset_states()
            
            # Training DataSet
            for x_np, y_np in TrainDataSet:
                with tf.GradientTape() as tape:
                    
                    # Use the model function to build the forward pass.
                    scores = model(x_np, training=is_training)
                    loss = loss_fn(y_np, scores)
      
                    gradients = tape.gradient(loss, model.trainable_variables)
                    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
                    
                    # Update the metrics
                    train_loss.update_state(loss)
                    train_accuracy.update_state(y_np, scores)
                    
                    if t % print_every == 0:
                        val_loss.reset_states()
                        val_accuracy.reset_states()
                        for test_x, test_y in ValidationDataSet:
                            # During validation at end of epoch, training set to False
                            prediction = model(test_x, training=False)
                            t_loss = loss_fn(test_y, prediction)

                            val_loss.update_state(t_loss)
                            val_accuracy.update_state(test_y, prediction)
                        
                        template = 'Iteration {}, Epoch {}, Loss: {}, Accuracy: {}, Val Loss: {}, Val Accuracy: {}'
                        print (template.format(t, epoch+1,
                                             train_loss.result(),
                                             train_accuracy.result()*100,
                                             val_loss.result(),
                                             val_accuracy.result()*100))
                    t += 1

In [ ]:
# Hyper-parameters tuning
kernel_1 = (5,5)
kernel_2 = (3,3)
pooling = (2,2)
filter_size = 32
hidden_layer = 64
dropout = 0.3
num_classes = 43

# Configurations of batch printing
print_every = 1000
num_epochs = 10
learning_rate = 1e-3

In [ ]:
model = CNN_Model()

def model_init_fn():
    return CNN_Model()

In [ ]:
def optimizer_init_fn():
    return tf.keras.optimizers.Adam(learning_rate) 

In [ ]:
def saveModel():
    model.save('CNN_model.h5')

In [ ]:
DataSetImages, DataSetLabels = LoadTrainDataSet()
image_train, image_valid,label_train, label_valid=train_test_split(DataSetImages,DataSetLabels,stratify=y,test_size=0.33,random_state=42)
TrainDataSet = image_train, label_train
ValidationDataSet = image_valid, label_valid
train(model_init_fn, optimizer_init_fn, num_epochs=num_epochs, TrainDataSet,ValidationDataSet, is_training=True)
saveCNNModel()